In [1]:
!pip install albumentations

     |████████████████████████████████| 72 kB 440 kB/s eta 0:00:011
     |████████████████████████████████| 30.2 MB 12.3 MB/s eta 0:00:01
     |████████████████████████████████| 948 kB 19.6 MB/s eta 0:00:01
     |████████████████████████████████| 38.2 MB 14.6 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 20.2 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 19.9 MB/s eta 0:00:01
     |████████████████████████████████| 51.0 MB 15.6 MB/s eta 0:00:01     |███████████████▏                | 24.2 MB 14.0 MB/s eta 0:00:02
     |████████████████████████████████| 1.6 MB 16.7 MB/s eta 0:00:01
     |████████████████████████████████| 4.4 MB 25.9 MB/s eta 0:00:01
     |████████████████████████████████| 165 kB 18.1 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [2]:
import os

import numpy as np
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
import warnings

warnings.filterwarnings("ignore")

import albumentations
import matplotlib.pyplot as plt

%matplotlib inline
import sklearn
import tensorflow as tf
import tensorflow_addons as tfa
from PIL import Image
from sklearn.model_selection import (
    StratifiedKFold,
    StratifiedShuffleSplit,
    train_test_split,
)
from tensorflow import keras
from tensorflow.keras.applications.efficientnet import EfficientNetB4, EfficientNetB7
from tensorflow.keras.layers import (
    AveragePooling2D,
    AvgPool2D,
    Conv2D,
    Dense,
    Flatten,
    GlobalAveragePooling2D,
    MaxPooling2D,
)
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import notebook, tqdm

In [3]:
PATH = "/app/_data/"
BATCH_SIZE = 26
SEED = 37
IMAGE_SIZE = 456
NUM_CLASSES = 6
TRAIN_IMG_PATH = "/app/_data/456_npy/"
TEST_IMG_PATH = "/app/_data/test_images/"
feature_columns = [
    "complex",
    "frog_eye_leaf_spot",
    "healthy",
    "powdery_mildew",
    "rust",
    "scab",
]

In [4]:
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
wrong = [
    "ead085dfac287263.jpg",
    "95276ccd226ad933.jpg",
    "da8770e819d2696d.jpg",
    "cd3a1d64e6806eb5.jpg",
    "ccec54723ff91860.jpg",
]
df_labels = pd.read_csv("/app/_data/df_csv/labels_21_20.csv", index_col=[0])
df_labels = df_labels.query("image not in @wrong").reset_index(drop=True)
df_labels["image"] = df_labels["image"].str.replace(".jpg", ".npy")
df_labels = df_labels.sample(frac=1, random_state=SEED).reset_index(drop=True)
df_complex = df_labels.query('labels == "complex"').reset_index(drop=True)
df_without_complex = df_labels.query('labels != "complex"').reset_index(drop=True)

In [6]:
df_without_complex["image"]

0        9888ce2d3e363acd.npy
1        901f704f717817b9.npy
2        91673f856431749b.npy
3        f09047534b76db29.npy
4        9005d36127b0f77b.npy
                 ...         
18643          Train_1595.npy
18644    afa23b9c5aa958a8.npy
18645    a194c734b2996bad.npy
18646    86f636c5e84e23cc.npy
18647    8fbc33c7895a330c.npy
Name: image, Length: 18648, dtype: object

In [7]:
# for 380*380
transform = albumentations.Compose(
    [
        albumentations.CLAHE(p=0.1, clip_limit=(1, 1), tile_grid_size=(8, 8)),
        albumentations.OneOf(
            [
                albumentations.MotionBlur((3, 3)),
                albumentations.MedianBlur(blur_limit=3),
                albumentations.GaussianBlur(blur_limit=(3, 3), sigma_limit=0),
                albumentations.Blur(blur_limit=(3, 3)),
            ],
            p=0.2,
        ),
        albumentations.OneOf(
            [
                albumentations.GaussNoise(var_limit=[10, 20], mean=1),
                albumentations.ISONoise(
                    intensity=(0.1, 0.2), color_shift=(0.001, 0.005)
                ),
                albumentations.ImageCompression(
                    quality_lower=90, quality_upper=100, compression_type=1
                ),
                albumentations.MultiplicativeNoise(
                    multiplier=(0.98, 1.02), per_channel=True, elementwise=True
                ),
                albumentations.Downscale(
                    scale_min=0.9, scale_max=0.99, interpolation=4
                ),
            ],
            p=0.5,
        ),
        albumentations.OneOf(
            [
                albumentations.HueSaturationValue(
                    hue_shift_limit=(-5, 5),
                    sat_shift_limit=(-5, 5),
                    val_shift_limit=(-5, 5),
                ),
                albumentations.RandomBrightnessContrast(
                    brightness_limit=0.1,
                    contrast_limit=0.1,
                    brightness_by_max=True,
                ),
            ],
            p=0.5,
        ),
        albumentations.OneOf(
            [
                albumentations.OpticalDistortion(
                    distort_limit=0.05,
                    shift_limit=0.05,
                    border_mode=2,
                ),
                albumentations.ElasticTransform(
                    alpha=2.0,
                    sigma=50.0,
                    alpha_affine=10.0,
                    interpolation=0,
                    border_mode=2,
                ),
                albumentations.GridDistortion(
                    num_steps=5, distort_limit=0.1, interpolation=0, border_mode=2
                ),
            ],
            p=0.5,
        ),
        albumentations.OneOf(
            [
                albumentations.HorizontalFlip(),
                albumentations.VerticalFlip(),
            ],
            p=0.5,
        ),
        albumentations.OneOf(
            [
                albumentations.Rotate(
                    limit=(-180, 180), interpolation=0, border_mode=2
                ),
                albumentations.ShiftScaleRotate(
                    shift_limit=0.05,
                    scale_limit=0.05,
                    rotate_limit=180,
                    interpolation=0,
                    border_mode=2,
                ),
            ],
            p=0.5,
        ),
    ]
)

In [8]:
class Generator(keras.utils.Sequence):
    def __init__(
        self,
        df,
        images_src_dir,
        batch_size,
        target_image_size,
        shuffle=False,
        augment=True,
        crop=False,
        resize=False,
        normalize=False,
    ):
        self.shuffle = shuffle
        self.batch_size = BATCH_SIZE
        self.df = df
        self.images_dir = images_src_dir
        self.target_image_size = (IMAGE_SIZE, IMAGE_SIZE)
        self.augment = augment
        self.crop = crop
        self.resize = resize
        self.normalize = normalize
        # create label index map
        self.labels = self._read_labels()
        self.n_samples = self.df.shape[0]
        self.n_batches = self.n_samples // self.batch_size
        # shuffle data, also repeated after each epoch if needed
        if self.shuffle:
            np.random.shuffle(self.labels)

    def _read_labels(self):
        """
        Returns list images mapping to 1-hot label
        """

        # label indexes
        label_ixs = self.df[feature_columns].values
        image_ixs = self.df["image"].values
        labels = []

        for i in range(len(image_ixs)):
            labels.append([image_ixs[i], label_ixs[i]])
        return labels

    def __len__(self):
        """
        Length in batches
        """
        return self.n_batches

    def __getitem__(self, b_ix):
        """
        Produce batch, by batch index
        """

        assert b_ix < self.n_batches

        b_X = np.zeros(
            (self.batch_size, self.target_image_size[0], self.target_image_size[1], 3),
            dtype=np.uint8,
        )

        b_Y = np.zeros(
            (self.batch_size, self.df[feature_columns].shape[1]),
            dtype=np.uint8,
        )

        for i in range(self.batch_size):
            b_X[i], b_Y[i] = self.get_one(
                i + self.batch_size * b_ix,
            )

        return (b_X, b_Y)

    def get_one(self, one_ix):
        """
        Get single item by absolute index
        """
        id = self.labels[one_ix][0]
        src_file = self.images_dir + id

        # read file
        x = np.load(src_file)
        if self.crop:
            coord = self.df[self.df["image"] == id][
                ["x_min", "y_min", "x_max", "y_max"]
            ].values[0]
            orig_hight = x.shape[0]
            orig_width = x.shape[1]
            x_min = coord[0]
            y_min = coord[1]
            x_max = coord[2]
            y_max = coord[3]
            x = x[
                np.int(y_min * orig_hight) : np.int(y_max * orig_hight),
                np.int(x_min * orig_width) : np.int(x_max * orig_width),
            ]

        y = self.labels[one_ix][1]

        # augment
        if self.augment:
            x = self._augment_image(x)

        # normalize (sample-wise)
        if self.normalize:
            x = x.astype(np.float32)
            x = x - np.mean(x, axis=(0, 1))
            x = x / np.std(x, axis=(0, 1))
        return x.astype(np.uint8), y

    def _augment_image(self, x):
        """
        Randomply augment image
        """

        x = transform(image=x)["image"]
        return x

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.labels)

In [9]:
policy = keras.mixed_precision.experimental.Policy("mixed_float16")
keras.mixed_precision.experimental.set_policy(policy)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3090, compute capability 8.6


In [10]:
def get_model():
    inputs = keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
    base_model = keras.applications.EfficientNetB5(weights=None, include_top=False)
    base_model.load_weights(
        "/app/_data/models/efficientnet-b5_noisy-student_notop.h5",
        by_name=True,
        skip_mismatch=True,
    )
    x = base_model(inputs)
    x = keras.layers.GlobalAveragePooling2D(name="avg_pool")(x)
    x = keras.layers.Flatten(name="flatten")(x)
    outputs = keras.layers.Dense(NUM_CLASSES, activation="sigmoid")(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(
        loss="binary_crossentropy",
        optimizer=Adam(lr=0.0005),
        metrics=[
            "acc",
            keras.metrics.Recall(),
            keras.metrics.Precision(),
            tfa.metrics.F1Score(num_classes=NUM_CLASSES, average="weighted"),
        ],
    )
    return model

In [ ]:
skf = StratifiedKFold(n_splits=5, random_state=SEED, shuffle=True)

for i, (train_index, valid_index) in enumerate(
    skf.split(df_without_complex["image"], df_without_complex["labels"])
):
    train, valid = (
        df_without_complex.loc[train_index],
        df_without_complex.loc[valid_index],
    )
    model_name = "eff5_without_complex_" + str(i + 1) + ".h5"
    log_dir = "logs_eff5_wo_complex_" + str(i + 1) + "/"
    print('\nStart of #'+str(i+1)+' iteration\n')

    callbacks = [
        keras.callbacks.EarlyStopping(
            monitor="val_f1_score",
            patience=12,
            restore_best_weights=True,
            verbose=1,
            mode="max",
        ),
        keras.callbacks.ModelCheckpoint(
            "/app/_data/models/final/eff5WOComplex/" + model_name,
            monitor="val_f1_score",
            verbose=1,
            save_best_only=True,
            save_weights_only=False,
            mode="max",
            save_freq="epoch",
        ),
        keras.callbacks.ReduceLROnPlateau(
            monitor="val_f1_score",
            factor=0.9,
            patience=8,
            verbose=1,
            mode="max",
            min_delta=1e-4,
            min_lr=0.00000001,
        ),
        keras.callbacks.TensorBoard(
            log_dir="/app/.tensorboard/" + log_dir, histogram_freq=0
        ),
        keras.callbacks.experimental.BackupAndRestore(
            "/app/_data/models/final/eff5WOComplex/backup/"
        ),
        keras.callbacks.TerminateOnNaN(),
    ]
    model = get_model()
    gen_train = Generator(
        df=train,
        images_src_dir=TRAIN_IMG_PATH,
        target_image_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        shuffle=True,
        augment=True,
        crop=False,
        resize=False,
    )
    gen_valid = Generator(
        df=valid,
        images_src_dir=TRAIN_IMG_PATH,
        target_image_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        shuffle=False,
        augment=False,
        crop=False,
        resize=False,
    )

    history = model.fit(
        gen_train,
        validation_data=gen_valid,
        epochs=100,
        steps_per_epoch=train.shape[0] // BATCH_SIZE,
        validation_steps=valid.shape[0] // BATCH_SIZE,
        verbose=1,
        workers=25,
        callbacks=callbacks,
    )
    tf.keras.backend.clear_session()


Start of #1 iteration

Epoch 1/100
  1/573 [..............................] - ETA: 0s - loss: 0.7612 - acc: 0.1154 - recall: 0.5000 - precision: 0.1346 - f1_score: 0.1031WARNING:tensorflow:From /usr/local/lib/python3.8/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
573/573 [==============================] - ETA: 0s - loss: 0.1043 - acc: 0.8866 - recall: 0.8733 - precision: 0.9173 - f1_score: 0.8779
Epoch 00001: val_f1_score improved from -inf to 0.90456, saving model to /app/_data/models/final/eff5WOComplex/eff5_without_complex_1.h5
573/573 [==============================] - 384s 670ms/step - loss: 0.1043 - acc: 0.8866 - recall: 0.8733 - precision: 0.9173 - f1_score: 0.8779 - val_loss: 0.0840 - val_acc: 0.9250 - val_recall: 0.9059 - val_precision: 0.9342 - val_f1_score: 0.9046
Epoch 2/100
573/573 [========

In [11]:
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [10]:
def pred_to_labels(pred, thresh=0.5, labels=feature_columns):
    pred = [labels[i] for i in range(len(labels)) if pred[i] > thresh]
    pred = " ".join(pred)
    return pred


def predict_new(path, model):
    img = np.load(TRAIN_IMG_PATH + path)
    img = tf.expand_dims(img, axis=0)
    pred = model.predict(img)
    return pred_to_labels(pred[0])

In [19]:
model_list = [
    "eff4_ns_cr_complex_kf_v2_3.h5",
    "eff4_ns_cr_complex_kf_v2_2.h5",
    "eff4_ns_cr_complex_kf_v2_5.h5",
    "eff4_ns_cr_complex_kf_v2_4.h5",
    "eff4_ns_cr_complex_kf_v2_1.h5",
]

In [20]:
df_sub = pd.DataFrame(columns=["image", "labels"])
for model_path in model_list:
    model = keras.models.load_model(
        "/app/_data/models/complex/pred_complex_2/" + model_path
    )
    for img_name in df_complex["image"].tolist():
        pred = predict_new(img_name, model)

        df_sub = df_sub.append({"image": img_name, "labels": pred}, ignore_index=True)

    print(df_sub.head())

                  image                      labels
0  a99471cdb22da5e2.npy          frog_eye_leaf_spot
1  8780b695d39cdf81.npy          frog_eye_leaf_spot
2  e917184f15c81adf.npy     frog_eye_leaf_spot scab
3  f55a824bda558e25.npy     frog_eye_leaf_spot scab
4  d5b2c6c99f58b007.npy  complex frog_eye_leaf_spot
                  image                      labels
0  a99471cdb22da5e2.npy          frog_eye_leaf_spot
1  8780b695d39cdf81.npy          frog_eye_leaf_spot
2  e917184f15c81adf.npy     frog_eye_leaf_spot scab
3  f55a824bda558e25.npy     frog_eye_leaf_spot scab
4  d5b2c6c99f58b007.npy  complex frog_eye_leaf_spot
                  image                      labels
0  a99471cdb22da5e2.npy          frog_eye_leaf_spot
1  8780b695d39cdf81.npy          frog_eye_leaf_spot
2  e917184f15c81adf.npy     frog_eye_leaf_spot scab
3  f55a824bda558e25.npy     frog_eye_leaf_spot scab
4  d5b2c6c99f58b007.npy  complex frog_eye_leaf_spot
                  image                      labels
0  a99471cdb

In [11]:
df_sub = df_sub.join(df_sub["labels"].str.get_dummies(" "))

NameError: name 'df_sub' is not defined

In [ ]:
df_sub.shape[0] / 5

In [12]:
len(df_sub[df_sub["complex"] == 1]) / 5

NameError: name 'df_sub' is not defined

In [13]:
# df_sub.to_csv("/app/_data/df_csv/complex_predicted_kf_main.csv")
df_sub = pd.read_csv("/app/_data/df_csv/complex_predicted_kf_main.csv", index_col=[0])

In [14]:
# df_sub['complex']=1

In [15]:
df_sub

,image,labels,complex,frog_eye_leaf_spot,healthy,powdery_mildew,rust,scab
0,a99471cdb22da5e2.npy,frog_eye_leaf_spot,0,1,0,0,0,0
1,8780b695d39cdf81.npy,frog_eye_leaf_spot,0,1,0,0,0,0
2,e917184f15c81adf.npy,frog_eye_leaf_spot scab,0,1,0,0,0,1
3,f55a824bda558e25.npy,frog_eye_leaf_spot scab,0,1,0,0,0,1
4,d5b2c6c99f58b007.npy,complex frog_eye_leaf_spot,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...
7880,bf22f63ed33ac010.npy,complex rust,1,0,0,0,1,0
7881,ca1be18029db0bfe.npy,complex rust,1,0,0,0,1,0
7882,bf76e8d3c0c045c5.npy,frog_eye_leaf_spot,0,1,0,0,0,0
7883,d03fe072c99b26cc.npy,scab,0,0,0,0,0,1


In [16]:
df_pivot = df_sub.pivot_table(index="image", aggfunc=sum)

In [17]:
df_pivot

,complex,frog_eye_leaf_spot,healthy,powdery_mildew,rust,scab
image,,,,,,
800cbf0ff87721f8.npy,3,5,0,0,0,5
801f78399a44e7af.npy,4,5,0,0,0,4
802291cee9fec9f4.npy,1,5,0,0,0,5
803e3bd17a16e65c.npy,5,5,0,0,0,2
8052c7272b3b3f5c.npy,0,5,0,0,0,4
...,...,...,...,...,...,...
ffb084c28f4f30cc.npy,3,5,0,0,0,5
ffd4c5088f4605a7.npy,2,5,0,0,0,5
ffdc70c492370f02.npy,0,2,0,0,4,2


In [18]:
df_pivot[df_pivot <= 4] = 0

In [19]:
df_pivot[df_pivot > 4] = 1

In [20]:
df_pivot["complex"] = 1

In [21]:
for i in df_pivot.index.tolist():
    pred = pred_to_labels(df_pivot.loc[i].values)
    df_pivot.loc[i, "labels"] = pred

In [22]:
df_pivot.sample()

,complex,frog_eye_leaf_spot,healthy,powdery_mildew,rust,scab,labels
image,,,,,,,
e313d3c48594f1dc.npy,1,1,0,0,0,1,complex frog_eye_leaf_spot scab


In [23]:
df_pivot["labels"].value_counts()

complex frog_eye_leaf_spot         658
complex frog_eye_leaf_spot scab    451
complex rust                       153
complex scab                       152
complex                            132
complex powdery_mildew              22
complex frog_eye_leaf_spot rust      5
complex healthy                      4
Name: labels, dtype: int64

In [24]:
df_pivot["image"] = df_pivot.index
df_pivot = df_pivot.reset_index(drop=True)

In [25]:
df_all = pd.concat([df_without_complex, df_pivot], axis=0, ignore_index=True)

In [26]:
df_all = df_all.sample(frac=1)

In [27]:
df_all

,image,labels,complex,frog_eye_leaf_spot,healthy,powdery_mildew,rust,scab
17315,d1b1b3a336e3044f.npy,scab,0,0,0,0,0,1
8600,ac29c3b15dd2611f.npy,scab,0,0,0,0,0,1
662,81f497994a696f16.npy,scab frog_eye_leaf_spot,0,1,0,0,0,1
3577,a4dc8f289e77502d.npy,healthy,0,0,1,0,0,0
17590,9797b1d058e38dc1.npy,scab,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...
17085,adc384d1deaa9a19.npy,rust,0,0,0,0,1,0
16528,bd0b90e51a87e1ce.npy,healthy,0,0,1,0,0,0
2089,Train_1808.npy,healthy,0,0,1,0,0,0
4934,Train_1789.npy,healthy,0,0,1,0,0,0


In [28]:
tf.keras.backend.clear_session()

# StratifiedShuffleSplit

In [29]:
policy = keras.mixed_precision.experimental.Policy("mixed_float16")
keras.mixed_precision.experimental.set_policy(policy)

In [30]:
def get_model():
    inputs = keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
    base_model = keras.applications.EfficientNetB4(weights=None, include_top=False)
    base_model.load_weights(
        "/app/_data/models/efficientnet-b4_noisy-student_notop.h5",
        by_name=True,
        skip_mismatch=True,
    )
    x = base_model(inputs)
    x = keras.layers.GlobalAveragePooling2D(name="avg_pool")(x)
    x = keras.layers.Flatten(name="flatten")(x)
    outputs = keras.layers.Dense(NUM_CLASSES, activation="sigmoid")(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(
        loss="binary_crossentropy",
        optimizer=Adam(lr=0.0005),
        metrics=[
            "acc",
            keras.metrics.Recall(),
            keras.metrics.Precision(),
            tfa.metrics.F1Score(num_classes=NUM_CLASSES, average="weighted"),
        ],
    )
    return model

In [31]:
skf = StratifiedKFold(n_splits=5, random_state=SEED, shuffle=True)

for i, (train_index, valid_index) in enumerate(
    skf.split(df_all["image"], df_all["labels"])
):
    if i >= 4:
        train, valid = df_all.loc[train_index], df_all.loc[valid_index]
        model_name = (
            "eff4_ns_cr_with_predicted_complex_v2_kf_final_" + str(i + 1) + ".h5"
        )
        log_dir = "logs_complex_pred_" + str(i + 1) + "/"

        callbacks = [
            keras.callbacks.EarlyStopping(
                monitor="val_f1_score",
                patience=12,
                restore_best_weights=True,
                verbose=1,
                mode="max",
            ),
            keras.callbacks.ModelCheckpoint(
                "/app/_data/models/complex/pred_complex_2/" + model_name,
                monitor="val_f1_score",
                verbose=1,
                save_best_only=True,
                save_weights_only=False,
                mode="max",
                save_freq="epoch",
            ),
            keras.callbacks.ReduceLROnPlateau(
                monitor="val_f1_score",
                factor=0.8,
                patience=5,
                verbose=1,
                mode="max",
                min_delta=1e-4,
                min_lr=0.00000001,
            ),
            keras.callbacks.TensorBoard(
                log_dir="/app/.tensorboard/logs4/", histogram_freq=0
            ),
        ]
        model = get_model()
        gen_train = Generator(
            df=train,
            images_src_dir=TRAIN_IMG_PATH,
            target_image_size=IMAGE_SIZE,
            batch_size=BATCH_SIZE,
            shuffle=True,
            augment=True,
            crop=False,
            resize=False,
        )
        gen_valid = Generator(
            df=valid,
            images_src_dir=TRAIN_IMG_PATH,
            target_image_size=IMAGE_SIZE,
            batch_size=BATCH_SIZE,
            shuffle=False,
            augment=False,
            crop=False,
            resize=False,
        )

        history = model.fit(
            gen_train,
            validation_data=gen_valid,
            epochs=100,
            steps_per_epoch=train.shape[0] // BATCH_SIZE,
            validation_steps=valid.shape[0] // BATCH_SIZE,
            verbose=1,
            workers=15,
            #         use_multiprocessing=True,
            callbacks=callbacks,
        )
        keras.backend.clear_session()

Epoch 1/100
  1/288 [..............................] - ETA: 0s - loss: 0.6982 - acc: 0.1786 - recall: 0.5079 - precision: 0.1951 - f1_score: 0.2253WARNING:tensorflow:From /usr/local/lib/python3.8/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
288/288 [==============================] - ETA: 0s - loss: 0.1306 - acc: 0.8232 - recall: 0.8488 - precision: 0.8914 - f1_score: 0.8170
Epoch 00001: val_f1_score improved from -inf to 0.82951, saving model to /app/_data/models/complex/pred_complex_2/eff4_ns_cr_with_predicted_complex_v2_kf_final_5.h5
288/288 [==============================] - 143s 497ms/step - loss: 0.1306 - acc: 0.8232 - recall: 0.8488 - precision: 0.8914 - f1_score: 0.8170 - val_loss: 0.1077 - val_acc: 0.8447 - val_recall: 0.8605 - val_precision: 0.9272 - val_f1_score: 0.8295
Epoch 2/100
288/288 [====

In [ ]:
"cd3a1d64e6806eb5.jpg", "ccec54723ff91860.jpg", ""

In [1]:
import kaggle

In [2]:
! kaggle datasets init -p /app/_data/models/complex/pred_complex_2/withoutComplex/

Data package template written to: /app/_data/models/complex/pred_complex_2/withoutComplex/dataset-metadata.json


In [3]:
! kaggle datasets create -p /app/_data/models/complex/pred_complex_2/withoutComplex/

Starting upload for file eff4_ns_cr_complex_kf_v2_3.h5
100%|█████████████████████████████████████████| 203M/203M [05:07<00:00, 692kB/s]
Upload successful: eff4_ns_cr_complex_kf_v2_3.h5 (203MB)
Starting upload for file eff4_ns_cr_complex_kf_v2_2.h5
100%|█████████████████████████████████████████| 203M/203M [05:04<00:00, 698kB/s]
Upload successful: eff4_ns_cr_complex_kf_v2_2.h5 (203MB)
Starting upload for file eff4_ns_cr_complex_kf_v2_5.h5
100%|█████████████████████████████████████████| 203M/203M [05:04<00:00, 700kB/s]
Upload successful: eff4_ns_cr_complex_kf_v2_5.h5 (203MB)
Starting upload for file eff4_ns_cr_complex_kf_v2_4.h5
100%|█████████████████████████████████████████| 203M/203M [05:04<00:00, 699kB/s]
Upload successful: eff4_ns_cr_complex_kf_v2_4.h5 (203MB)
Starting upload for file eff4_ns_cr_complex_kf_v2_1.h5
100%|█████████████████████████████████████████| 203M/203M [05:05<00:00, 698kB/s]
Upload successful: eff4_ns_cr_complex_kf_v2_1.h5 (203MB)
Skipping folder: .ipynb_checkpoints